In [982]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

In [983]:
matches = pd.read_csv('matches.csv', index_col = 0) # read in our scraped data

In [984]:
matches.head() # next few lines explore our data

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,dist,sh,sot,fk,pk,pkatt,season,team
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,Match Report,NaN,14.8,18.0,5.0,0.0,0,0,2025,Liverpool
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,Match Report,NaN,13.6,19.0,8.0,1.0,0,0,2025,Liverpool
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,Match Report,NaN,13.4,11.0,3.0,0.0,0,0,2025,Liverpool
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,Match Report,NaN,14.9,14.0,5.0,0.0,0,0,2025,Liverpool
5,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3.0,0.0,Bournemouth,...,Match Report,NaN,16.6,19.0,12.0,0.0,0,0,2025,Liverpool


In [985]:
matches.shape

(2400, 28)

In [986]:
matches.team.value_counts()

team
Liverpool                   120
Brighton and Hove Albion    120
Wolverhampton Wanderers     120
Crystal Palace              120
Everton                     120
Manchester City             120
Manchester United           120
Brentford                   120
West Ham United             120
Chelsea                     120
Tottenham Hotspur           120
Newcastle United            120
Arsenal                     120
Aston Villa                 120
Bournemouth                  82
Southampton                  82
Nottingham Forest            82
Leicester City               82
Fulham                       82
Burnley                      76
Leeds United                 76
Luton Town                   38
Sheffield United             38
Watford                      38
Norwich City                 38
Ipswich Town                  6
Name: count, dtype: int64

In [987]:
matches.dtypes

date              object
time              object
comp              object
round             object
day               object
venue             object
result            object
gf               float64
ga               float64
opponent          object
xg               float64
xga              float64
poss             float64
attendance       float64
captain           object
formation         object
opp formation     object
referee           object
match report      object
notes            float64
dist             float64
sh               float64
sot              float64
fk               float64
pk                 int64
pkatt              int64
season             int64
team              object
dtype: object

In [988]:
matches.date = pd.to_datetime(matches.date) # date column to datetime dtype

In [ ]:
# Format columns with dtype object into numeric dtypes to be used for ML

In [989]:
matches['venue'] = matches['venue'].astype('category').cat.codes

In [990]:
matches['hour'] = matches['time'].str.replace(':[0-6][0-9]', '', regex = True).astype(int)

In [991]:
matches['day'] = matches['date'].dt.dayofweek

In [992]:
matches['target'] = (matches['result'] == 'W').astype('int')

In [993]:
# format into numeric dtypes with dictionary of codes and corresponding labels
team_cols = ['team', 'opponent', 'formation', 'opp formation']
team_list = []
for col in team_cols:
    matches[col] = matches[col].astype('category')
    team_list.append(dict(enumerate(matches[col].cat.categories)))
    matches[col] = matches[col].cat.codes
for idx, name in enumerate(team_cols):
    print(name)
    print(team_list[idx])

team
{0: 'Arsenal', 1: 'Aston Villa', 2: 'Bournemouth', 3: 'Brentford', 4: 'Brighton and Hove Albion', 5: 'Burnley', 6: 'Chelsea', 7: 'Crystal Palace', 8: 'Everton', 9: 'Fulham', 10: 'Ipswich Town', 11: 'Leeds United', 12: 'Leicester City', 13: 'Liverpool', 14: 'Luton Town', 15: 'Manchester City', 16: 'Manchester United', 17: 'Newcastle United', 18: 'Norwich City', 19: 'Nottingham Forest', 20: 'Sheffield United', 21: 'Southampton', 22: 'Tottenham Hotspur', 23: 'Watford', 24: 'West Ham United', 25: 'Wolverhampton Wanderers'}
opponent
{0: 'Arsenal', 1: 'Aston Villa', 2: 'Bournemouth', 3: 'Brentford', 4: 'Brighton', 5: 'Burnley', 6: 'Chelsea', 7: 'Crystal Palace', 8: 'Everton', 9: 'Fulham', 10: 'Ipswich Town', 11: 'Leeds United', 12: 'Leicester City', 13: 'Liverpool', 14: 'Luton Town', 15: 'Manchester City', 16: 'Manchester Utd', 17: 'Newcastle Utd', 18: 'Norwich City', 19: "Nott'ham Forest", 20: 'Sheffield Utd', 21: 'Southampton', 22: 'Tottenham', 23: 'Watford', 24: 'West Ham', 25: 'Wolv

In [994]:
matches['round'] = matches['round'].astype('category').cat.codes

In [995]:
team_codes_list = team_list[0]

In [996]:
opponent_codes_list = team_list[1]

In [997]:
formation_codes_list = team_list[2]

In [998]:
opp_formation_codes_list = team_list[3]

In [999]:
matches = matches.drop(columns = ['time', 'result', 'notes', 'comp', 'captain', 'referee', 'match report'])

In [1000]:
matches.dtypes

date             datetime64[ns]
round                      int8
day                       int32
venue                      int8
gf                      float64
ga                      float64
opponent                   int8
xg                      float64
xga                     float64
poss                    float64
attendance              float64
formation                  int8
opp formation              int8
dist                    float64
sh                      float64
sot                     float64
fk                      float64
pk                        int64
pkatt                     int64
season                    int64
team                       int8
hour                      int64
target                    int64
dtype: object

In [1001]:
matches.head()

,date,round,day,venue,gf,ga,opponent,xg,xga,poss,...,dist,sh,sot,fk,pk,pkatt,season,team,hour,target
0,2024-08-17,0,5,0,2.0,0.0,10,2.6,0.5,62.0,...,14.8,18.0,5.0,0.0,0,0,2025,13,12,1
1,2024-08-25,11,6,1,2.0,0.0,3,2.5,0.5,62.0,...,13.6,19.0,8.0,1.0,0,0,2025,13,16,1
2,2024-09-01,22,6,0,3.0,0.0,16,1.8,1.4,47.0,...,13.4,11.0,3.0,0.0,0,0,2025,13,16,1
3,2024-09-14,32,5,1,0.0,1.0,19,0.9,0.4,68.0,...,14.9,14.0,5.0,0.0,0,0,2025,13,15,0
5,2024-09-21,33,5,1,3.0,0.0,2,2.0,1.1,58.0,...,16.6,19.0,12.0,0.0,0,0,2025,13,15,1


In [1002]:
# create rolling averages of past 3 games to use as predictors and improve accuracy
def rolling_averages(group, cols, new_cols):
    group = group.sort_values(by='date')
    rolling_data = group[cols].rolling(3, closed = 'left').mean()
    group[new_cols] = rolling_data
    group = group.dropna(subset = new_cols)
    return group

In [1003]:
# create rolling averages of past 3 games against the specific opponent to use as predictors and improve accuracy
def rolling_averages_vs_opponent(data, cols):
    matches_new = matches.copy()
    matches_new['matchup'] = matches_new.apply(
        lambda x: f"{min([x['team'], x['opponent']])} vs {max([x['team'], x['opponent']])}", 
        axis=1
    )
    matches_new['matchup'] = matches_new['matchup'].astype('category')
    matches_new['matchup'] = matches_new['matchup'].cat.codes

   
    matches_new = matches_new.sort_values(by=['matchup', 'date']).reset_index(drop=True)

    for col in cols:
        matches_new[f'rolling_avg_{col}_vs_opp'] = matches_new.groupby('team')[col].rolling(3, closed='left').mean().reset_index(level=0, drop=True)
    
    final_df = data.merge(matches_new[['date'] + [f'rolling_avg_{col}_vs_opp' for col in cols]], on='date', how='left')
    return final_df

In [ ]:
# create new prdictors in next few lines

In [1004]:
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols = [f'{col}_rolling' for col in cols]

In [1005]:
matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))

/var/folders/q3/c8j9rrxj6vx_zl07g75p5xpc0000gn/T/ipykernel_72254/1214488446.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))


In [1006]:
matches_rolling

date  round  day  venue   gf   ga  opponent   xg  xga  poss  ...  \
team                                                                      ...   
0    4 2021-09-11     32    5      1  1.0  0.0        18  2.7  0.7  51.0  ...   
     5 2021-09-18     33    5      0  1.0  0.0         5  1.1  1.0  54.0  ...   
     7 2021-09-26     34    6      1  3.0  1.0        22  1.1  1.0  46.0  ...   
     8 2021-10-02     35    5      0  0.0  0.0         4  0.4  1.1  42.0  ...   
     9 2021-10-18     36    0      1  2.0  2.0         7  1.7  0.7  54.0  ...   
...           ...    ...  ...    ...  ...  ...       ...  ...  ...   ...  ...   
25   1 2024-08-25     11    6      1  2.0  6.0         6  1.9  1.6  40.0  ...   
     3 2024-08-31     22    5      0  1.0  1.0        19  0.7  1.0  48.0  ...   
     4 2024-09-15     32    6      1  1.0  2.0        17  1.2  1.5  49.0  ...   
     6 2024-09-21     33    5      0  1.0  3.0         1  0.5  1.7  47.0  ...   
     7 2024-09-28     34    5      1  1.0  2.0        13  0.6  2.5  45.0  ...   

        hour  target  gf_rolling  ga_rolling  sh_rolling  sot_rolling  \
team                                                                    
0    4    15       1    0.000000    3.000000    9.666667     2.333333   
     5    15       1    0.333333    2.333333   12.333333     3.000000   
     7    16       1    0.666667    1.666667   14.666667     3.000000   
     8    17       0    1.666667    0.333333   18.333333     5.333333   
     9    20       0    1.333333    0.333333   11.000000     4.000000   
...      ...     ...         ...         ...         ...          ...   
25   1    14       0    0.333333    2.333333    9.000000     4.333333   
     3    15       0    0.666667    3.333333    8.333333     3.000000   
     4    16       0    1.000000    3.000000   10.666667     3.000000   
     6    15       0    1.333333    3.000000   11.666667     3.666667   
     7    17       0    1.000000    2.000000   11.000000     3.666667   

        dist_rolling  fk_rolling  pk_rolling  pkatt_rolling  
team                                                         
0    4     14.833333    0.333333         0.0            0.0  
     5     14.133333    0.333333         0.0            0.0  
     7     14.800000    0.666667         0.0            0.0  
     8     18.433333    0.666667         0.0            0.0  
     9     19.833333    0.666667         0.0            0.0  
...              ...         ...         ...            ...  
25   1     17.633333    0.666667         0.0            0.0  
     3     19.133333    1.000000         0.0            0.0  
     4     19.700000    0.666667         0.0            0.0  
     6     18.333333    0.333333         0.0            0.0  
     7     16.933333    0.000000         0.0            0.0  

[2322 rows x 31 columns]

In [1007]:
matches_rolling = matches_rolling.droplevel('team')

In [1008]:
matches_complete = rolling_averages_vs_opponent(matches_rolling, cols)

In [1009]:
matches_complete

,date,round,day,venue,gf,ga,opponent,xg,xga,poss,...,pk_rolling,pkatt_rolling,rolling_avg_gf_vs_opp,rolling_avg_ga_vs_opp,rolling_avg_sh_vs_opp,rolling_avg_sot_vs_opp,rolling_avg_dist_vs_opp,rolling_avg_fk_vs_opp,rolling_avg_pk_vs_opp,rolling_avg_pkatt_vs_opp
0,2021-09-11,32,5,1,1.0,0.0,18,2.7,0.7,51.0,...,0.0,0.0,2.000000,0.666667,14.000000,4.666667,17.033333,0.000000,0.000000,0.000000
1,2021-09-11,32,5,1,1.0,0.0,18,2.7,0.7,51.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-11,32,5,1,1.0,0.0,18,2.7,0.7,51.0,...,0.0,0.0,1.000000,3.333333,8.000000,3.000000,16.866667,0.333333,0.333333,0.333333
3,2021-09-11,32,5,1,1.0,0.0,18,2.7,0.7,51.0,...,0.0,0.0,2.333333,1.333333,18.666667,7.000000,16.566667,0.333333,0.333333,0.333333
4,2021-09-11,32,5,1,1.0,0.0,18,2.7,0.7,51.0,...,0.0,0.0,3.000000,0.666667,20.666667,7.000000,16.966667,0.666667,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22311,2024-09-28,34,5,1,1.0,2.0,13,0.6,2.5,45.0,...,0.0,0.0,1.666667,2.000000,18.666667,7.000000,17.033333,0.333333,0.000000,0.000000
22312,2024-09-28,34,5,1,1.0,2.0,13,0.6,2.5,45.0,...,0.0,0.0,1.333333,1.000000,12.333333,4.333333,17.766667,0.333333,0.000000,0.000000
22313,2024-09-28,34,5,1,1.0,2.0,13,0.6,2.5,45.0,...,0.0,0.0,1.000000,1.333333,11.333333,2.666667,16.900000,0.000000,0.333333,0.333333
22314,2024-09-28,34,5,1,1.0,2.0,13,0.6,2.5,45.0,...,0.0,0.0,2.666667,1.000000,16.000000,4.000000,14.866667,0.666667,0.000000,0.000000


In [1010]:
matches_complete = matches_complete.sort_values(by='date').reset_index(drop=True)

In [1011]:
# drop any null values incurred in the creation of new predictors
matches_complete = matches_complete.dropna(subset=[f'rolling_avg_{col}_vs_opp'for col in cols])

In [1012]:
matches_complete

,date,round,day,venue,gf,ga,opponent,xg,xga,poss,...,pk_rolling,pkatt_rolling,rolling_avg_gf_vs_opp,rolling_avg_ga_vs_opp,rolling_avg_sh_vs_opp,rolling_avg_sot_vs_opp,rolling_avg_dist_vs_opp,rolling_avg_fk_vs_opp,rolling_avg_pk_vs_opp,rolling_avg_pkatt_vs_opp
0,2021-09-11,32,5,1,1.0,0.0,18,2.7,0.7,51.0,...,0.000000,0.000000,2.000000,0.666667,14.000000,4.666667,17.033333,0.000000,0.000000,0.000000
1,2021-09-11,32,5,0,0.0,1.0,0,0.7,2.7,49.0,...,0.333333,0.333333,1.666667,0.333333,12.000000,3.333333,19.200000,1.000000,0.000000,0.000000
2,2021-09-11,32,5,1,0.0,0.0,24,0.7,0.7,42.0,...,0.333333,0.333333,2.000000,0.666667,14.000000,4.666667,17.033333,0.000000,0.000000,0.000000
4,2021-09-11,32,5,1,0.0,0.0,24,0.7,0.7,42.0,...,0.333333,0.333333,1.000000,3.333333,8.000000,3.000000,16.866667,0.333333,0.333333,0.333333
5,2021-09-11,32,5,1,0.0,0.0,24,0.7,0.7,42.0,...,0.333333,0.333333,2.333333,1.333333,18.666667,7.000000,16.566667,0.333333,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22311,2024-09-29,34,6,0,2.0,2.0,10,0.8,1.2,56.0,...,0.000000,0.000000,1.000000,0.666667,10.666667,2.666667,18.533333,0.666667,0.000000,0.000000
22312,2024-09-30,34,0,1,3.0,1.0,21,1.3,0.6,40.0,...,0.000000,0.333333,0.333333,0.666667,8.333333,2.000000,17.533333,0.000000,0.000000,0.000000
22313,2024-09-30,34,0,1,3.0,1.0,21,1.3,0.6,40.0,...,0.000000,0.333333,1.000000,1.000000,20.666667,5.333333,17.366667,0.333333,0.000000,0.333333
22314,2024-09-30,34,0,0,1.0,3.0,2,0.6,1.3,60.0,...,0.000000,0.333333,0.333333,0.666667,8.333333,2.000000,17.533333,0.000000,0.000000,0.000000


In [1013]:
matches_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21713 entries, 0 to 22315
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      21713 non-null  datetime64[ns]
 1   round                     21713 non-null  int8          
 2   day                       21713 non-null  int32         
 3   venue                     21713 non-null  int8          
 4   gf                        21713 non-null  float64       
 5   ga                        21713 non-null  float64       
 6   opponent                  21713 non-null  int8          
 7   xg                        21713 non-null  float64       
 8   xga                       21713 non-null  float64       
 9   poss                      21713 non-null  float64       
 10  attendance                21673 non-null  float64       
 11  formation                 21713 non-null  int8          
 12  opp formation          

In [1014]:
matches_complete.index = range(matches_complete.shape[0])

In [1015]:
# create function to train and use our model
def make_predictions(data, predictors):
    train = data[data['date'] <= '2024-01-01']
    test = data[data['date'] > '2024-01-01']
    rf = RandomForestClassifier(n_estimators = 100, min_samples_split = 5, random_state = 0)
    rf.fit(train[predictors], train['target'])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test['target'], predicted = preds), index = test.index)
    precision = precision_score(test['target'], preds)
    return combined, precision

In [1016]:
predictors = matches_complete.columns[~matches_complete.columns.isin(['target', 'date', 'attendance', 'matchup', 'dist', 'gf', 'ga', 'xg', 'xga', 'poss', 'sh', 'sot', 'fk', 'pk', 'pkatt'])]

In [1017]:
predictors

Index(['round', 'day', 'venue', 'opponent', 'formation', 'opp formation',
       'season', 'team', 'hour', 'gf_rolling', 'ga_rolling', 'sh_rolling',
       'sot_rolling', 'dist_rolling', 'fk_rolling', 'pk_rolling',
       'pkatt_rolling', 'rolling_avg_gf_vs_opp', 'rolling_avg_ga_vs_opp',
       'rolling_avg_sh_vs_opp', 'rolling_avg_sot_vs_opp',
       'rolling_avg_dist_vs_opp', 'rolling_avg_fk_vs_opp',
       'rolling_avg_pk_vs_opp', 'rolling_avg_pkatt_vs_opp'],
      dtype='object')

In [1018]:
combined, precision = make_predictions(matches_complete, predictors)

In [1019]:
precision

0.5972323379461034

In [1020]:
matches_complete.to_csv('final_data.csv')